In [ ]:
import numpy as np
import pickle
import scipy.ndimage
import matplotlib.pyplot as plt
import math

In [ ]:
def jet_fly_lawnmower(n_loops = 5, ymin = 0.0, ymax = 20.0, v0 = 2.0, Ts = 0.2, w = 5.0*math.pi/180.0, sigmaLik = 0.005, sigma = 0.05):
    
    #X = [x1 x1' x2 x2'] aircraft position in arbitrary coord. frame
    
    #Constant Velocity (CV)
    Acv = np.identity(4)
    Acv[0,1] = Acv[2,3] = Ts

    #Coordinated turn ccw
    w = -w;
    Acw = np.identity(4)
    Acw[0,1] = Acw[2,3] = math.sin(w*Ts)/w
    Acw[0,3] =  (1-math.cos(w*Ts))/w
    Acw[2,1] = - Acw[0,3]
    Acw[1,1] = Acw[3,3] = math.cos(w*Ts)
    Acw[1,3] = -math.sin(w*Ts)
    Acw[3,1] = -Acw[1,3]
    
    #coordinated turn ccw
    w = -w;
    Accw = np.identity(4)
    Accw[0,1] = Accw[2,3] = math.sin(w*Ts)/w
    Accw[0,3] =  (1-math.cos(w*Ts))/w
    Accw[2,1] = - Accw[0,3]
    Accw[1,1] = Accw[3,3] = math.cos(w*Ts)
    Accw[1,3] = -math.sin(w*Ts)
    Accw[3,1] = -Accw[1,3]

    print "Model CV: A is \n"+str(Acv)
    print "Model CCW: A is \n"+str(Accw)
    print "Model CW: A is \n"+str(Acw)
    
    m = 0
    lm = 'up'
    A = Acv
    delta = v0*0.001
    
    X = np.array([0,0,ymin,v0])
    
    X_ = X
    loop_cnt = 0
    ms = np.array([m])
    while loop_cnt < n_loops:
        X = np.dot(A, X)
        X_ = np.vstack((X_,X))
        ms = np.vstack((ms,[m]))
        if(X[2] >= ymax and lm == 'up'):
            lm = 'cw'
            A = Acw
            m = 1
        if(X[3] <= -(v0-delta) and lm == 'cw'):
            lm = 'down'
            A = Acv
            m = 0
        if(X[2] <= ymin and lm == 'down'):
            lm = 'ccw'
            A = Accw
            m = 2
        if(X[3] >= v0-delta and lm == 'ccw'):
            lm = 'up'
            A = Acv
            m = 0 
            loop_cnt += 1    
    X_ += np.random.randn(*X_.shape)*sigma
    return X_,ms

track, ms = jet_fly_lawnmower(n_loops =1)

In [ ]:
def make_inputs(start,end,input_type):
    interval = end-start
    data = np.ones((interval,1))
    
    if input_type in ['cw','cv']:
        dat = np.ones((interval,1))
        for ii in range(interval):
            dat[ii,0] = ii
            
        data = np.hstack((data,dat))
    if input_type in ['cw']:
        dat = np.ones((interval,1))
        for ii in range(interval):
            dat[ii,0] = ii*ii
            
        data = np.hstack((data,dat))
        
    return data

def make_inputs2(start,end,input_type):
    interval = end-start
    data = np.ones((interval,1))
    dat = track[(start-1):(end-1),[0,2]]
    data = np.hstack((data,dat))
    if input_type in ['cw']:
        dat = track[(start-1):(end-1),[1,3]]
        data = np.hstack((data,dat))
        
    return data

In [ ]:
import statsmodels.api as sm

i = 60
j = 200
plt.plot(track[i:j,0],track[i:j,2],'bo')
        
plt.axis('equal')
axis = 2
X = make_inputs(i,j,'cv')
ols = sm.GLS(track[i:j,axis],X,hasconst=False)
fit = ols.fit()
Y_hat = ols.predict(fit.params)
plt.plot(track[i:j,0],Y_hat,'rx')




X = make_inputs(i,j,'cw')
ols = sm.GLS(track[i:j,axis],X,hasconst=False)
fit = ols.fit()
Y_hat = ols.predict(fit.params)
plt.plot(track[i:j,0],Y_hat,'gx')
print ols.information(fit.params)

plt.show()

In [ ]:
%%time
import statsmodels.api as sm



likes = {}

print "Points:",len(track)

# Takahashi Meijin constant, 60 frames / 16 inputs ~= 4 frames per input.
# But note that in general transitions may happen more frequently due to collisions, etc.
min_interval = 1 


step = 10
axis = 0

js = []

for i in range(1,len(track),step):
    likes[i] = {}
    min_likelihood = float('inf')
    print i
    for j in range(i+step,len(track),step):
        js.append(j)
        models = {}
        for model in ['cv','cw']:
            models[model] = {}
            for axis in [0,2]:
                X = make_inputs(i,j,model) #make_inputs(i,j,model)
                Y = track[i:j,axis]
                ols = sm.OLS(Y,X,hasconst=False)
                results = ols.fit()
                models[model][axis] = results
        likes[i][j] = models

#print ''

In [ ]:
i = 71
j =161

print likes[i][j]['cv'][2].summary(),likes[i][j]['cw'][2].summary()
plt.plot(track[i:j,0],likes[i][j]['cv'][2].model.predict(likes[i][j]['cv'][2].params))
plt.show()

In [ ]:
modes
modes = {}
cost_weight = 16
modes[1] = (0,None)
for j in js:
    modes[j] = (0,None)
    least = float("inf")
    least_template = None
    for i in range(1, j, step):
        for modeltype in likes[i][j]:
            data = likes[i][j][modeltype]
            if data:
                crit = 0
                for axis,model in data.items():
                    crit_ = model.bic
                    if False:
                        if crit_ < 0:
                            #print 'A',(j-i),np.log(j-i)*len(model.params)
                            crit += -.05*(j-i) + np.log(j-i)*len(model.params)**2
                        else:

                            crit += -2*model.model.loglike(model.params)+ np.log(j-i)*len(model.params)**2
                    else:
                        crit += crit_

                
                
                #np.log(j-i)*cost_weight*len(model.params)
                cost = np.log(len(track))*len(model.params)*2  
                m_prev = modes[i][0]      
                here = crit + m_prev + cost
                #print i, crit, cost,here
                if here < least:
                    least = here
                    # prev_i,this_j,t0,t1,name,summary,criterion
                    least_template = (i,j,(modeltype,likes[i][j][modeltype]))
                    
    modes[j] = (least, least_template)

In [ ]:
print modes[461]
def get_path(modes):
    mj = sorted(modes)[-1]
    path = [modes[mj]]
    while mj > 1:
        print mj
        mj = modes[mj][1][0]
        path.append(modes[mj])
    return list(reversed(path))[1:]

path = get_path(modes)
for ii,p in enumerate(path):
    print ii,p[0],'\n',p[1][0],p[1][1],p[1][2][1][0].params,p[1][2][1][2].params,'\n'
    plt.plot(track[p[1][0]:p[1][1],0],track[p[1][0]:p[1][1],2])
plt.show()


mode_dat = np.zeros( track.shape[0])
for ii,p in enumerate(path):
    mode_dat[p[1][0]:(p[1][1]-2)] = len(p[1][2][1][0].params)
    
for ind in range(len(mode_dat)):
    if mode_dat[ind] ==0:
        mode_dat[ind]  = mode_dat[ind-1]
ms[ms > 1] = 1
plt.plot(mode_dat-2,'r')
print np.sum(np.abs(ms[:,0] - (mode_dat-2)))/float(len(mode_dat))

plt.plot(ms)
plt.show()

In [ ]:
cross = {}

for ii,mode in enumerate(path):
    model1 = mode[1][2][1]
    model1_type = mode[1][2][0]
    model1_params = model1.params
    model1 = model1.model
    for jj,mode2 in enumerate(path):
        model2 = mode2[1][2][1]
        model2_type = mode2[1][2][0]
        model2_params = model2.params
        model2 = model2.model
        if model1_type == model2_type:
            crit = -model2.loglike(model1_params)
            if crit == float('-inf'):
                crit = -1*(all_times[mode[1][1]]-all_times[mode[1][0]])
        else:
            crit = float('inf')
        cross[(ii,jj)] = crit

In [ ]:
"""UnionFind.py

Union-find data structure. Based on Josiah Carlson's code,
http://aspn.activestate.com/ASPN/Cookbook/Python/Recipe/215912
with significant additional changes by D. Eppstein.
"""

class UnionFind:
    """Union-find data structure.

    Each unionFind instance X maintains a family of disjoint sets of
    hashable objects, supporting the following two methods:

    - X[item] returns a name for the set containing the given item.
      Each set is named by an arbitrarily-chosen one of its members; as
      long as the set remains unchanged it will keep the same name. If
      the item is not yet part of a set in X, a new singleton set is
      created for it.

    - X.union(item1, item2, ...) merges the sets containing each item
      into a single larger set.  If any item is not yet part of a set
      in X, it is added to X as one of the members of the merged set.
    """

    def __init__(self):
        """Create a new empty union-find structure."""
        self.weights = {}
        self.parents = {}

    def __getitem__(self, object):
        """Find and return the name of the set containing the object."""

        # check for previously unknown object
        if object not in self.parents:
            self.parents[object] = object
            self.weights[object] = 1
            return object

        # find path of objects leading to the root
        path = [object]
        root = self.parents[object]
        while root != path[-1]:
            path.append(root)
            root = self.parents[root]

        # compress the path and return
        for ancestor in path:
            self.parents[ancestor] = root
        return root
        
    def __iter__(self):
        """Iterate through all items ever found or unioned by this structure."""
        return iter(self.parents)

    def union(self, *objects):
        """Find the sets containing the objects and merge them all."""
        roots = [self[x] for x in objects]
        heaviest = max([(self.weights[r],r) for r in roots])[1]
        for r in roots:
            if r != heaviest:
                self.weights[heaviest] += self.weights[r]
                self.parents[r] = heaviest

In [ ]:
cost_weight = 2
unions = UnionFind()
for d in sorted(cross):
    
    t1 = all_times[path[d[0]][1][1]]-all_times[path[d[0]][1][0]]
    t2 = all_times[path[d[1]][1][1]]-all_times[path[d[1]][1][0]]
    complexityWeight = cost_weight * (np.log(t1*t2)*len(path[d[0]][1][2][1].params))
    good = True
    for t in [cross[d], cross[(d[0],d[0])], cross[(d[1],d[1])], cross[(d[1],d[0])]]:
        if t == float('inf'):
            good = False
    if not good:
        continue
    joinedWeight = cost_weight * (np.log(t1+t2)*len(path[d[0]][1][2][1].params))
    #print path[d[0]][1][2][1].params,path[d[1]][1][2][1].params
    #print min(cross[d] + cross[(d[0],d[0])],cross[(d[1],d[1])]+cross[(d[1],d[0])]),joinedWeight, (cross[(d[0],d[0])]+cross[(d[1],d[1])]) , complexityWeight 
    
    joined = min(cross[d] + cross[(d[0],d[0])],cross[(d[1],d[1])]+cross[(d[1],d[0])]) +joinedWeight
    
    if (joined <= (cross[(d[0],d[0])]+cross[(d[1],d[1])]) +complexityWeight):
        unions.union(d[0],d[1])
        
merged = {}
for u in unions:
    if unions[u] not in merged:
        merged[unions[u]] = set()
    merged[unions[u]].add(u)
print len(merged)
for m in merged:
    print m, merged[m]

In [ ]:
all_times2 = all_times[:60]
plt.plot(velocities[all_times2[0]:all_times2[-1]])
plt.plot(np.array(all_times2)-all_times2[0],velocities[np.array(all_times2,dtype='int')],'rx')

m2i = {m:i for i,m in enumerate(merged)}
print m2i
colors = ['r','g','b','c','m','y','k','#ff8800','#0088ff','#ff0088','#88ff00','#00ff88','#8800ff']
merged_params = {}
for m in merged:
    models = sorted(merged[m])
    sub = models[0]
    interval = path[sub][1][1]-path[sub][1][0]
    params =  path[sub][1][2][1].params*interval
    
    total = interval
    for sub in models[1:]:
        interval = path[sub][1][1]-path[sub][1][0]
        params +=  path[sub][1][2][1].params*interval
        total += interval
    merged_params[m] = params/float(total)

for u in sorted(unions):
    t0 = all_times[path[u][1][0]]-all_times2[0]
    t1 = all_times[path[u][1][1]]-all_times2[0]
    u = m2i[unions[u]]
    if t0 < all_times2[-1]-all_times2[0] and t1 >= 0:
        print t0, t1, u,merged_params[unions[u]]
        plt.plot([t0,t1],[u+5,u+5])#,colors[u])
        
        
plt.show()
